# Decision trees

Welcome to the first jupyter notebook! In this session, we won't go into too many minute details yet, but will cover the basics of machine learning. We've tried to keep it as simple as possible, also because many might only have little experience with python and/or programming a 'learning machine'. If it looks like you're gonna be through the content of this notebook in ten minutes or so, because you're already familiar with all of its concepts, then feel free to challenge yourself a little more with the overarching machine-learning challenge.

## Setup

To allow the next code blocks to run smoothly, this section sets a couple of settings.

Some imports that we will be using:

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

Set the random seed to a fixed number. This will guarantee that the notebook output is generated the same way for every run, otherwise the seed would be – random, as the name suggests.

In [ ]:
np.random.seed(42)

Some figure plotting settings: increase the axis labels of our figures a bit.

In [ ]:
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

Before we start, we'll define the following function to show decision boundaries in our plots. Ignore the details of this implementation ...

In [ ]:
from matplotlib.colors import ListedColormap

def plot_decision_boundary(clf, X, y, axes=[-1.5, 2.5, -1, 1.5]):
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo")
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs")
    plt.plot(X[:, 0][y==2], X[:, 1][y==2], "g^")
    plt.axis(axes)
    plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    plt.xlabel(r"$x_1$", fontsize=18)
    plt.ylabel(r"$x_2$", fontsize=18, rotation=0)

## Classification trees

Let's start with an easy classification problem and see what can be done about it with decision trees. The first step is to generate some random data. The Scikit Learn package provides a couple of functions for these, so we won't have to worry about it ourselves. The function used is [make_moons](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html). Run the following cell to import the function and generate 100 samples of two classes in half-moon shape.

In [ ]:
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=100, noise=0.25, random_state=53)

We can examine the structure of our data as we did in the previous notebook:

In [ ]:
X[0:3]

Remember: this command outputs the feature vectors of the first three instances. From what we can tell above, every of these instances has two features (the square brackets mark the beginning/the end of an entry in the overarching vector). Let's also have a look at the target values:

In [ ]:
y[0:3]

Look's like a typical classification problem: our entries are either 0's, meaning that this instance belongs to "class 0", or 1's for instances of "class 1". Since our dataset is reasonably small with only 100 instances, we should maybe just plot it to get an idea what we're talking about (ignore the details of these plotting commands):

In [ ]:
plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo")
plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs")
plt.axis([-1.5, 2.5, -1, 1.5])
plt.xlabel(r"$x_1$", fontsize=18)
plt.ylabel(r"$x_2$", fontsize=18, rotation=0)
plt.show()

Ok, this indeed looks like two half moons, or something like a yin-and-yang structure. If we used something like a linear regression, we would definitely not be able to fit this dataset very well, but maybe a decision tree could help us with that?

Let's define a decision tree! For that, we use the [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) class of Scikit Learn, that comes with lots of functionalities useful for us.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X, y)

From the output, you can already see that the model comes with a bunch of parameters that we could have set by hand. Many of these have very resonable default values, such as `criterion='gini'`. Just a quick reminder of the Gini impurity index:

$$ G_i = 1 - \sum_{k=1}^n p_{i,k}^2 $$

for classes $k=1,\dots,n$ and leaf $i$ of a tree. We could also switch to using entropy as a measure when/where/how often to split the decision tree.

Before we discuss all of these parameters in a bit more detail, let's just use the "default" decision-tree classifier and see how well it performs:

In [ ]:
plot_decision_boundary(tree, X, y)
plt.show()

Hmm. The model seems to produce quite a few small "boxes" with only very few, sometimes even just one instance in it. Would you say this model looks particularly great? Would it generalise well on a new dataset generated from the same shape?

Maybe it would be useful to apply some sort of regularisation to the model. As we're extremely sensitive to outliers at the moment, maybe we could require a minimum number of instances for each leaf of the tree.

In [ ]:
tree = DecisionTreeClassifier(min_samples_leaf=4, random_state=42)
tree.fit(X, y)

Let's create the same plot as before:

In [ ]:
plot_decision_boundary(tree, X, y)
plt.show()

This definitely does look a lot better, doesn't it? There are a few outliers that the decision boundary doesn't catch, but probably – on a completely new dataset – this model would perform a lot better than the one above. We usually say: _it generalises well with unseen data_. There are many more regularisation parameters, feel free to try them out:
* Maximum number of features considered: in our case we only have two features anyways, but in many cases this makes sense.
* Maximum number of leaves in the entire tree.
* Minimum decrease of impurity (in our case, the Gini impurity) when moving from one level to the next.
* Minimum impurity value to still allow further splits.
* Minimum number of instances required for a split.
* ...

## Bagging trees

In [ ]:
from sklearn.model_selection import train_test_split
X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_tree))

In [ ]:
from sklearn.ensemble import BaggingClassifier
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1, random_state=42)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

In [ ]:
plt.figure(figsize=(11,4))
plt.subplot(121)
plot_decision_boundary(tree_clf, X, y)
plt.title("Decision Tree", fontsize=14)
plt.subplot(122)
plot_decision_boundary(bag_clf, X, y)
plt.title("Decision Trees with Bagging", fontsize=14)
plt.show()

## Adaptive boosting

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, y_train)

In [ ]:
y_pred = ada_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

In [ ]:
plot_decision_boundary(ada_clf, X, y)

# Regression trees

In [ ]:
# Quadratic training set + noise
np.random.seed(42)
m = 200
X = np.random.rand(m, 1)
y = 4 * (X - 0.5) ** 2
y = y + np.random.randn(m, 1) / 10

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg.fit(X, y)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(random_state=42, max_depth=2)
tree_reg2 = DecisionTreeRegressor(random_state=42, max_depth=3)
tree_reg1.fit(X, y)
tree_reg2.fit(X, y)

def plot_regression_predictions(tree_reg, X, y, axes=[0, 1, -0.2, 1], ylabel="$y$"):
    x1 = np.linspace(axes[0], axes[1], 500).reshape(-1, 1)
    y_pred = tree_reg.predict(x1)
    plt.axis(axes)
    plt.xlabel("$x_1$", fontsize=18)
    if ylabel:
        plt.ylabel(ylabel, fontsize=18, rotation=0)
    plt.plot(X, y, "b.")
    plt.plot(x1, y_pred, "r.-", linewidth=2, label=r"$\hat{y}$")

plt.figure(figsize=(11, 4))
plt.subplot(121)
plot_regression_predictions(tree_reg1, X, y)
for split, style in ((0.1973, "k-"), (0.0917, "k--"), (0.7718, "k--")):
    plt.plot([split, split], [-0.2, 1], style, linewidth=2)
plt.text(0.21, 0.65, "Depth=0", fontsize=15)
plt.text(0.01, 0.2, "Depth=1", fontsize=13)
plt.text(0.65, 0.8, "Depth=1", fontsize=13)
plt.legend(loc="upper center", fontsize=18)
plt.title("max_depth=2", fontsize=14)

plt.subplot(122)
plot_regression_predictions(tree_reg2, X, y, ylabel=None)
for split, style in ((0.1973, "k-"), (0.0917, "k--"), (0.7718, "k--")):
    plt.plot([split, split], [-0.2, 1], style, linewidth=2)
for split in (0.0458, 0.1298, 0.2873, 0.9040):
    plt.plot([split, split], [-0.2, 1], "k:", linewidth=1)
plt.text(0.3, 0.5, "Depth=2", fontsize=13)
plt.title("max_depth=3", fontsize=14)

plt.show()

In [ ]:
tree_reg1 = DecisionTreeRegressor(random_state=42)
tree_reg2 = DecisionTreeRegressor(random_state=42, min_samples_leaf=10)
tree_reg1.fit(X, y)
tree_reg2.fit(X, y)

x1 = np.linspace(0, 1, 500).reshape(-1, 1)
y_pred1 = tree_reg1.predict(x1)
y_pred2 = tree_reg2.predict(x1)

plt.figure(figsize=(11, 4))

plt.subplot(121)
plt.plot(X, y, "b.")
plt.plot(x1, y_pred1, "r.-", linewidth=2, label=r"$\hat{y}$")
plt.axis([0, 1, -0.2, 1.1])
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", fontsize=18, rotation=0)
plt.legend(loc="upper center", fontsize=18)
plt.title("No restrictions", fontsize=14)

plt.subplot(122)
plt.plot(X, y, "b.")
plt.plot(x1, y_pred2, "r.-", linewidth=2, label=r"$\hat{y}$")
plt.axis([0, 1, -0.2, 1.1])
plt.xlabel("$x_1$", fontsize=18)
plt.title("min_samples_leaf={}".format(tree_reg2.min_samples_leaf), fontsize=14)

plt.show()